In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [10]:
SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
ALL_PROB_FILE = 'SpamData/03_Testing/prob-all.txt'

TEST_FEATURE_FILE = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

## Load the Data

In [11]:
## Features
X_test = np.loadtxt(TEST_FEATURE_FILE, delimiter = ' ')
## Target
y_test = np.loadtxt(TEST_TARGET_FILE, delimiter=' ')

## Token Probabilities
prob_spam = np.loadtxt(SPAM_PROB_FILE, delimiter = ' ')
prob_ham = np.loadtxt(HAM_PROB_FILE, delimiter = ' ')
prob_all = np.loadtxt(ALL_PROB_FILE, delimiter = ' ')

In [12]:
prob_spam.shape

(2500,)

In [13]:
X_test.dot(prob_spam).shape

(1723,)